In [1]:
from importlib.metadata import version
import torch

print("torch version:", version("torch"))

torch version: 2.5.1


In [2]:
# We take an example from the embedding vector on CH2, we take for instance [:,:, 0] the first embedding dimension

inputsCh2= torch.tensor(
    [[ 1.4117,  0.9599,  1.8852,  2.5063],
     [ 1.3674,  1.0696,  0.3401,  1.3563],
     [ 1.5433,  0.0562,  1.0025,  1.7163],
     [ 2.8333,  4.8586,  0.3163,  1.8257],
     [ 1.1971,  0.2007,  2.7901,  5.2530],
     [ 1.8696,  2.4908,  1.9542,  2.2539],
     [ 0.1828, -0.1302, -1.8858,  0.5482],
     [-1.2937,  1.7937,  0.4240,  3.5123]]
)

In [3]:
attn_scores = torch.empty(8, 8) # The wij

attn_scores = inputsCh2 @ inputsCh2.T
print(attn_scores)

tensor([[12.7498,  6.9975,  8.4241, 13.8356, 20.3081, 14.3632, -2.0481,  9.4977],
        [ 6.9975,  4.9690,  4.8392, 11.6548,  9.9251,  8.9422,  0.2129,  5.0575],
        [ 8.4241,  4.8392,  6.3356,  8.0962, 13.6716,  8.8528, -0.6748,  4.5575],
        [13.8356, 11.6548,  8.0962, 35.0668, 14.8398, 22.1320,  0.2897, 11.5959],
        [20.3081,  9.9251, 13.6716, 14.8398, 36.8520, 20.0302, -2.1892, 18.4444],
        [14.3632,  8.9422,  8.8528, 22.1320, 20.0302, 18.5985, -2.4322, 10.7940],
        [-2.0481,  0.2129, -0.6748,  0.2897, -2.1892, -2.4322,  3.9071,  0.6558],
        [ 9.4977,  5.0575,  4.5575, 11.5959, 18.4444, 10.7940,  0.6558, 17.4070]])


In [4]:
attn_weights = torch.softmax(attn_scores, dim=-1) # Alphas normalized by softmax
print(attn_weights)

tensor([[5.1933e-04, 1.6491e-06, 6.8676e-06, 1.5381e-03, 9.9531e-01, 2.6070e-03,
         1.9445e-10, 2.0093e-05],
        [7.5521e-03, 9.9337e-04, 8.7240e-04, 7.9559e-01, 1.4110e-01, 5.2802e-02,
         8.5413e-06, 1.0852e-03],
        [5.1676e-03, 1.4335e-04, 6.4015e-04, 3.7231e-03, 9.8228e-01, 7.9336e-03,
         5.7766e-07, 1.0815e-04],
        [6.0172e-10, 6.7965e-11, 1.9356e-12, 1.0000e+00, 1.6425e-09, 2.4126e-06,
         7.8794e-16, 6.4081e-11],
        [6.5324e-08, 2.0222e-12, 8.5677e-11, 2.7556e-10, 1.0000e+00, 4.9473e-08,
         1.1082e-17, 1.0132e-08],
        [3.6700e-04, 1.6231e-06, 1.4843e-06, 8.6815e-01, 1.0611e-01, 2.5351e-02,
         1.8643e-11, 1.0341e-05],
        [2.3411e-03, 2.2456e-02, 9.2430e-03, 2.4250e-02, 2.0330e-03, 1.5945e-03,
         9.0311e-01, 3.4972e-02],
        [9.5983e-05, 1.1320e-06, 6.8658e-07, 7.8247e-04, 7.3743e-01, 3.5090e-04,
         1.3875e-08, 2.6134e-01]])


In [5]:
# We can verify that the rows indeed all sum to 1
print("All row sums:", attn_weights.sum(dim=-1))

All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [6]:
all_context_vecs = attn_weights @ inputsCh2
print(all_context_vecs)

tensor([[ 1.2014,  0.2143,  2.7836,  5.2384],
        [ 2.5337,  4.0356,  0.7644,  2.3383],
        [ 1.2096,  0.2403,  2.7678,  5.1992],
        [ 2.8333,  4.8586,  0.3163,  1.8257],
        [ 1.1971,  0.2007,  2.7901,  5.2530],
        [ 2.6347,  4.3028,  0.6209,  2.2005],
        [ 0.2422,  0.0941, -1.6505,  0.7287],
        [ 0.5477,  0.6215,  2.1694,  4.7941]])


In [11]:
#  scaled dot-product attention
import torch.nn as nn

class SelfAttention_V2(nn.Module):
    ''' 
    torch.manual_seed(789)
    sa_v2 = SelfAttention_v2(d_in, d_out)
    print(sa_v2(inputs))
    '''
    def __init__(self, d_in, d_out, qkv_bias = False):

        super().__init__()

        self.Wq = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.Wk = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.Wv = nn.Linear(d_in, d_out, bias = qkv_bias)
    
    def forward(self, x):
        queries = self.Wq(x)
        keys = self.Wk(x)
        values = self.Wv(x)
        # operations
        att_scores = queries @ keys.T 
        attn_weights = torch.softmax(att_scores / keys.shape[-1]**0.5, dim= -1)
        
        context_vecs = attn_weights @ values

        return context_vecs 

In [12]:
# Test Class
d_in = 4
d_out = 2

torch.manual_seed(123)
sa_v2 = SelfAttention_V2(d_in, d_out)
print(sa_v2(inputsCh2))

tensor([[0.4550, 2.2955],
        [0.3292, 2.1056],
        [0.4204, 2.2676],
        [0.2697, 1.9142],
        [0.6549, 2.3685],
        [0.3765, 2.2369],
        [0.4498, 1.6045],
        [0.4838, 1.8963]], grad_fn=<MmBackward0>)


In [20]:
# Causal Attention

queries = sa_v2.Wq(inputsCh2)
keys = sa_v2.Wk(inputsCh2)
values = sa_v2.Wv(inputsCh2)

attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores/ keys.shape[-1]**0.5, dim = -1)
print(attn_weights)

context_length = attn_scores.shape[0]
# Pytorch tril function to create a mask where the values above the main diagonal are zero
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[8.2122e-02, 4.7738e-02, 4.5904e-02, 5.1775e-01, 9.1729e-02, 1.9330e-01,
         7.0099e-03, 1.4448e-02],
        [1.2226e-01, 9.6412e-02, 9.4113e-02, 2.7915e-01, 1.2901e-01, 1.7898e-01,
         4.1605e-02, 5.8469e-02],
        [9.3145e-02, 5.8019e-02, 5.6150e-02, 4.6264e-01, 1.0247e-01, 1.9634e-01,
         1.0879e-02, 2.0350e-02],
        [1.3428e-01, 1.2272e-01, 1.2790e-01, 1.5957e-01, 1.3187e-01, 1.4822e-01,
         8.9548e-02, 8.5893e-02],
        [2.2233e-02, 8.5820e-03, 7.0030e-03, 8.1758e-01, 2.9909e-02, 1.1275e-01,
         2.9361e-04, 1.6459e-03],
        [1.0608e-01, 6.8767e-02, 6.9904e-02, 4.0463e-01, 1.1175e-01, 2.0137e-01,
         1.4895e-02, 2.2597e-02],
        [9.8875e-02, 1.1299e-01, 1.0125e-01, 8.7562e-02, 1.0537e-01, 8.9213e-02,
         1.7968e-01, 2.2505e-01],
        [9.9814e-02, 9.7181e-02, 7.7770e-02, 2.0187e-01, 1.1874e-01, 1.2710e-01,
         8.7120e-02, 1.9041e-01]], grad_fn=<SoftmaxBackward0>)
tensor([[ 2.8919,    -inf,    -inf,    -inf,    -in

In [29]:
class MultiHeadAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias = False):
        super().__init__()

        assert (d_out % num_heads == 0), \
            "d_out must be divisible by number of heads"

        self.d_out = d_out # Equivalent to embed_dim in Pytorch implementatio
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Exact division
        self.Wq = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.Wk = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.Wv = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask",
                             torch.triu(torch.ones(context_length, context_length),diagonal=1)
                             )
    
    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.Wk(x) # Shape: (b, num_tokens, d_out)
        queries = self.Wq(x)
        values = self.Wv(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) # Reshape with .view
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [69]:
torch.manual_seed(123)
batch = torch.stack((inputsCh2, inputsCh2), dim=0) # We simulate a batch by stacking two inputs repeated
print(batch.shape)

batch_size, context_length, d_in = batch.shape
d_out = 4
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[ 0.0265, -0.1521, -1.0080, -0.8497],
         [-0.0876, -0.1204, -0.7893, -0.5933],
         [-0.0988, -0.1205, -0.7365, -0.6083],
         [-0.2470, -0.0985, -0.9677, -0.2169],
         [-0.2313, -0.0917, -0.7651, -0.2627],
         [-0.2168, -0.0998, -0.9231, -0.2842],
         [-0.0011, -0.1784, -0.9655, -0.8397],
         [-0.0086, -0.1668, -0.5994, -0.6655]],

        [[ 0.0265, -0.1521, -1.0080, -0.8497],
         [-0.0876, -0.1204, -0.7893, -0.5933],
         [-0.0988, -0.1205, -0.7365, -0.6083],
         [-0.2470, -0.0985, -0.9677, -0.2169],
         [-0.2313, -0.0917, -0.7651, -0.2627],
         [-0.2168, -0.0998, -0.9231, -0.2842],
         [-0.0011, -0.1784, -0.9655, -0.8397],
         [-0.0086, -0.1668, -0.5994, -0.6655]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 8, 4])


In [73]:
# Comparison with Pytorch MHA
num_heads=2
embed_dim = d_in # for this example
mha_pytorch = nn.MultiheadAttention(embed_dim, num_heads)
attn_output, attn_output_weights = mha_pytorch(batch,batch,batch)

print(attn_output)
print("context_vecs.shape:", attn_output.shape)

tensor([[[-0.0798, -0.7021, -1.3452,  0.2773],
         [ 0.3470, -0.1250, -0.8812,  0.1149],
         [ 0.2979, -0.5315, -1.1052,  0.0736],
         [ 0.4598,  0.2329, -1.4461,  0.5127],
         [-0.0597, -1.0536, -2.2134,  0.2522],
         [-0.1633, -0.5807, -1.3920,  0.4599],
         [ 0.8749,  0.6854, -0.1404, -0.2742],
         [-0.2957,  0.3489, -0.6921,  0.2081]],

        [[-0.0798, -0.7021, -1.3452,  0.2773],
         [ 0.3470, -0.1250, -0.8812,  0.1149],
         [ 0.2979, -0.5315, -1.1052,  0.0736],
         [ 0.4598,  0.2329, -1.4461,  0.5127],
         [-0.0597, -1.0536, -2.2134,  0.2522],
         [-0.1633, -0.5807, -1.3920,  0.4599],
         [ 0.8749,  0.6854, -0.1404, -0.2742],
         [-0.2957,  0.3489, -0.6921,  0.2081]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 8, 4])
